In [1]:
%pip install wandb datasets transformers trl torch peft

Note: you may need to restart the kernel to use updated packages.


In [2]:
import wandb
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from trl import SFTTrainer, SFTConfig
import torch
from peft import LoraConfig, get_peft_model

In [4]:
# 모델과 토크나이저 로드
model_name = "google/gemma-3-1b-it"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [15]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# corpus.json 파일 읽기
with open('./data/corpus.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# train/test 분리 (80:20 비율)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

print(f"전체 데이터 크기: {len(data)}")
print(f"학습 데이터 크기: {len(train_data)}")
print(f"테스트 데이터 크기: {len(test_data)}")

# 데이터 예시 출력
print("\n=== 학습 데이터 예시 ===")
print(train_data[0])
print("\n=== 테스트 데이터 예시 ===")
print(test_data[0])

# 리스트를 Dataset 형식으로 변환
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)

전체 데이터 크기: 108
학습 데이터 크기: 86
테스트 데이터 크기: 22

=== 학습 데이터 예시 ===
{'input': '감정: 외로움 / 원하는 효과: 공감받고 싶다 / 직업: 프리랜서 / 현재 상황: 잠들기 전 휴식 / 집중도: 가볍게 읽을 수 있는 책', 'instruction': '외로움을 공감하며 위로받을 수 있는 3권을 선정, 추천 이유와 효과를 각각 기술하세요.', 'output': '1. 외로움의 습격 - 김만권\n\n* 추천 이유: 외로움의 본질과 현실을 있는 그대로 바라보고, 저자의 통찰과 사례를 통해 공감을 느낄 수 있습니다.\n* 이 책이 도움이 될 수 있는 이유: 외로움이 정상이자 성장의 동기가 될 수 있음을 일깨워줍니다.\n\n2. 혼자가 좋지만 고독사는 걱정입니다 - 몬가 미오코\n\n* 추천 이유: 혼자 사는 삶에 대한 여러 고민을 담아 유연하게 받아들이는 힘을 선사합니다.\n* 이 책이 도움이 될 수 있는 이유: 혼자와 고독에 대한 두려움을 덜고, 자신만의 행복을 느낄 수 있습니다.\n\n3. (쇼펜하우어의) 고독한 행복 - 아르투어 쇼펜하우어\n\n* 추천 이유: 철학자의 시선을 통해 고독을 긍정적인 감정으로 전환하고 싶을 때 적합합니다.\n* 이 책이 도움이 될 수 있는 이유: 철학적 사유와 실질적인 삶의 지혜를 함께 얻을 수 있습니다.'}

=== 테스트 데이터 예시 ===
{'input': '감정: 만족 / 원하는 효과: 새로운 시각과 영감 / 직업: 학생 / 상황: 주말 / 집중도: 깊이 몰입할 수 있는 책', 'instruction': '아래 상황에 맞는 3권을 선정, 추천 이유와 감정적 효과를 포함하여 작성.', 'output': '1. 각본 없음 - 아비 모건\n\n* 추천 이유: 단순한 이야기가 아닌 인생의 우연성과 변화를 직시할 수 있는 힘을 줍니다.\n* 이 책이 도움이 될 수 있는 이유: 자신의 삶을 주체적으로 바라보고 긍정하는 영감의 시간이 됩니다.\n\n2. 공감사회를 위한 담론들 - 강황선\n

In [14]:
# 프롬프트 포매팅 함수 정의
def formatting_func(example):
    # instruction, input, output이 단일 문자열인 경우를 처리
    instruction = example['instruction']
    input_text = example['input'].strip() if example['input'] else ""
    output = example['output']

    # 형식화된 프롬프트 생성
    text = f"[Instruction]\n{instruction.strip()}\n\n"
    if input_text:
        text += f"[Input]\n{input_text}\n\n"
    text += f"[Output]\n{output.strip()}"

    return text

# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal Language Modeling을 위해 False로 설정
)

In [ ]:
wandb.init(
    project='Hanghae99-book-recommandation',
    name=f'gpt-finetuning',
    reinit=True
)

model = AutoModelForCausalLM.from_pretrained(model_name)

trainer = SFTTrainer(
    model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    args=SFTConfig(
        output_dir=f"/tmp/clm-instruction-tuning",
        max_seq_length=128,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_steps=100,
        num_train_epochs=3,
        learning_rate=2e-5,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss"
    ),
    formatting_func=formatting_func,
    data_collator=data_collator,
)

# 학습 시작
train_result = trainer.train()
metrics = train_result.metrics
trainer.log_metrics("train", metrics)

trainer.save_model()

wandb.finish()

Applying formatting function to train dataset:   0%|          | 0/86 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/86 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/86 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/86 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/86 [00:00<?, ? examples/s]

...

Adding EOS to eval dataset:   0%|          | 0/22 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/22 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/22 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Epoch,Training Loss,Validation Loss
